In [2]:
import json
import elasticsearch
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [4]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import json

with open(input_data_folder+"food_user_qa_dataset.json",'rt') as f_in:
    data = json.load(f_in)

In [6]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dishi_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [7]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dishi_name': 'almond fudge banana cake'}

In [8]:
# Flatten the documents into a list of texts
texts = []
for dish in data['dishes']:
    for doc in dish["documents"]:
        texts.append(doc["text"])

In [1]:
#Step 2 create embeddings using pretrained models
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


C:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer("all-mpnet-base-v2")

# Create embeddings
embeddings = model.encode(texts)

# Save the embeddings along with their corresponding text
embedding_data = list(zip(texts, embeddings.tolist()))

# Save to a JSON file
with open('embeddings.json', 'w') as f:
    json.dump(embedding_data, f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nakka\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

C:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
len(model.encode("This is a simple sentence"))


384

In [15]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dishi_name': 'almond fudge banana cake',
 'text_vector': [0.03449180722236633,
  0.06103437766432762,
  -0.03340601176023483,
  0.04663167893886566,
  0.002620068145915866,
  0.01821102201938629,
  0.02113189734518528,
  -0.018273789435625076,
  0.05332517996430397,
  0.07002101093530655,
  0.003915894776582718,
  -0.13695941865444183,
  -0.08629322052001953,
  -0.13535600900650024,
  0.042913611978292465,
  -0.005807182751595974,
  0.04823851212859154,
  -0.06157195195555687,
  -2.105251587636303e-05,
  0.02553781494498253,
  0.0631554126739502,
  0.031545720994472504,
  0.00606145802885294,
  -0.05777645856142044,
  -0.02174616977572441,
  0.04484262689948082,
  -0.035916104912757874,
  -0.026758482679724693,
  -0.05808210372924805,
  -0.02171657606959343,
  -0.020915331318974495,
  0.018911145627498627,
  0.05785742029547691,
  0.0729

In [12]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [16]:
#Step 3: Setup ElasticSearch connection
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000028075F39E20>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [ ]:
"""
Step 4: Create Mappings and Index
Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

Each document is a collection of fields, which each have their own data type.

We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored
"""

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "text"},
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "dishi_name": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [ ]:
index_name = "dish-name"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [17]:
operations[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dishi_name': 'almond fudge banana cake',
 'text_vector': [0.03449180722236633,
  0.06103437766432762,
  -0.03340601176023483,
  0.04663167893886566,
  0.002620068145915866,
  0.01821102201938629,
  0.02113189734518528,
  -0.018273789435625076,
  0.05332517996430397,
  0.07002101093530655,
  0.003915894776582718,
  -0.13695941865444183,
  -0.08629322052001953,
  -0.13535600900650024,
  0.042913611978292465,
  -0.005807182751595974,
  0.04823851212859154,
  -0.06157195195555687,
  -2.105251587636303e-05,
  0.02553781494498253,
  0.0631554126739502,
  0.031545720994472504,
  0.00606145802885294,
  -0.05777645856142044,
  -0.02174616977572441,
  0.04484262689948082,
  -0.035916104912757874,
  -0.026758482679724693,
  -0.05808210372924805,
  -0.02171657606959343,
  -0.020915331318974495,
  0.018911145627498627,
  0.05785742029547691,
  0.0729

In [ ]:
#Step 5: Add documents into index
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
#Step 6: Create end user query

search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [ ]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [ ]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

In [ ]:
# Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)




In [43]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")


def generate_response_from_llm(search_results):
    prompt = "You are a helpful assistant. Answer the following question based on the information provided:\n\n"

    for result in search_results:
        prompt += f"Food item: {result['Food items']}\n"
        prompt += f"Ingredients: {result['ingredients']}\n"
        prompt += f"Price: {result['price']}\n"
        prompt += f"Calories: {result['calories perce']}%\n\n"

    prompt += "What can you tell me about the ingredients of these dishes?"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].text.strip()

# Example usage
query = "What are the ingredients in banana pudding cake?"
search_results = elastic_search(query)
llm_response = generate_response_from_llm(search_results)

print(llm_response)

ObjectApiResponse({'name': '81d6d96573fa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Knoc8xY-Teq-lYkTwu4Eww', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [44]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "category": {"type": "text"},
                    "dishes": {
                        "type": "nested",
                        "properties": {
                            "dish name": {"type": "text"},
                            "documents": {
                                "type": "nested",
                                "properties": {
                                    "id": {"type": "keyword"},
                                    "question": {"type": "text"},
                                    "section": {"type": "text"},
                                    "text": {"type": "text"}
                                }
                            }
                        }
                    }
                }
            }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [88]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "dish_name": {"type": "keyword"},
            "id": {"type": "keyword"}
        }
    }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [91]:
from tqdm.auto import tqdm
from elasticsearch.exceptions import NotFoundError, RequestError

for doc in tqdm(documents):
    try:
        es.index(index=index_name, document=doc)
    except (NotFoundError, RequestError) as e:
        print(f"Error indexing document: {doc}")
        print(e)


100%|██████████████████████████████████████████████████████████| 3584/3584 [03:17<00:00, 18.18it/s]


In [92]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "menu-items"

# Example search query
query = {
    "query": {
        "match": {
            "documents.text": "banana"
        }
    }
}

response = es.search(index=index_name, body=query)
print(response)


{'took': 1275, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [93]:
response = es.search(index=index_name, body={"query": {"match_all": {}}})
print(response)

{'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'menu-items', '_id': 'q1NncJEBM82uNR2Or_9y', '_score': 1.0, '_source': {'id': '3091.0_1', 'question': 'What are the ingredients?', 'section': 'ingredients', 'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']", 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rFNncJEBM82uNR2Or_-u', '_score': 1.0, '_source': {'id': '3091.0_2', 'question': 'How many calories does it have?', 'section': 'calories', 'text': 'The kitty litter cake has 354.2 calories.', 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rVNncJEBM82uNR2Or__h', '_score': 1.0, '_source': {'id': '3091.0_3

In [46]:
# Check index mapping
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

# Check index settings
settings = es.indices.get_settings(index=index_name)
print(settings)

{'menu-items': {'mappings': {'properties': {'dish_name': {'type': 'keyword'}, 'id': {'type': 'keyword'}, 'question': {'type': 'text'}, 'section': {'type': 'text'}, 'text': {'type': 'text'}}}}}
{'menu-items': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'menu-items', 'creation_date': '1724167714048', 'number_of_replicas': '0', 'uuid': 'LYmVzOZIRcqRaTQhnwAfZQ', 'version': {'created': '8040399'}}}}}


In [94]:
query = "almond fudge banana cake"

In [98]:
documents[0]

{'id': '142_1',
 'question': 'What are the ingredients?',
 'section': 'ingredients',
 'text': "The almond fudge banana cake contains the following ingredients: ['dole banana', 'sugar', 'margarine', 'eggs', 'amaretto liqueur', 'vanilla extract', 'all-purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'dole almond']",
 'dish_name': 'almond fudge banana cake'}

In [116]:
def elastic_search(query, dish_name=None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["dish_name^6", "question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add a filter by dish_name if it's provided
    if dish_name:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "dish_name.keyword": dish_name
            }
        }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)  # Collect all documents
        

    return result_docs


In [135]:
elastic_search(query)

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [132]:
def build_prompt(query, search_results):
    prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT provided.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [133]:
query

'what are ingredients in banana pudding cake contain'

In [134]:
search_results= elastic_search(query)
search_results

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [119]:
# if using OPENAI 
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [120]:
# iF USING groq
"""
from groq import Groq   

#create client calling Groq class

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

"""

'\nfrom groq import Groq   \n\n#create client calling Groq class\n\nclient = Groq(api_key=os.getenv(\'GROQ_API_KEY\'))\n\nfrom openai import OpenAI\nclient = OpenAI()\n\ndef llm(prompt):\n    response = client.chat.completions.create(\n        model="llama3-8b-8192",\n        messages=[{"role": "user", "content": prompt}]\n    )\n    \n    return response.choices[0].message.content\n\n'

In [150]:
query = 'dish name please'

def rag(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [151]:
rag(query)


"I'm sorry, the dish name you are asking for is baby food pineapple coconut carrot cake."